In [1]:
# import required module
import os
import cv2
# assign directory
dataset = []
data = dict()
directory = ['/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/images', '/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity/images']
labels = ['COVID','Normal','Pneumonia','Bacterial infection']
cnt = 0
# iterate over files in
# that directory
ids = 0
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)
        
        if os.path.isfile(f):
            im = cv2.imread(f)
            im = cv2.resize(im,(256,256))
            dataset.append([im,ids])
            cnt+=1
             
    data.update({labels[ids]: cnt})
    ids+=1


In [2]:
import matplotlib.pyplot as plt

def show_bar(D):

    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    # # for python 2.x:
    # plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
    # plt.xticks(range(len(D)), D.keys())  # in python 2.x

    plt.show()

# COVID-19 radiography dataset

In [2]:
import numpy as np
print(len(dataset))
x = []
y = []
for i in dataset:
    x.append(i[0])
    y.append(i[1])
x = np.array(x)
y = np.array(y)
x.shape

21165


(21165, 256, 256, 3)

# Imbalanced dataset training

In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
xtr, xts, ytr, yts = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)

In [5]:
xtr.shape

(16932, 256, 256, 3)

In [6]:
import torch
xtr = torch.from_numpy(xtr)
ytr = ytr.astype(int);
ytr = torch.from_numpy(ytr)

In [7]:
xts = torch.from_numpy(xts)
yts = yts.astype(int);
yts = torch.from_numpy(yts)

In [ ]:
xtr=xtr.float()
xts=xts.float()
ytr = ytr.float()
yts = yts.float()

In [23]:
import torch
from torch import *
my_dataset = torch.utils.data.TensorDataset(xtr,ytr) # create your datset
my_dataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32)

my_dataset = torch.utils.data.TensorDataset(xts,yts) # create your datset
my_tdataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32)

In [13]:

import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class lassifier(nn.Module): 
    def __init__(self): 
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) 
        self.pool = nn.MaxPool2d(2, 2) 
        self.dropout1 = nn.Dropout2d(0.25) 
        self.dropout2 = nn.Dropout2d(0.5) 
        self.fc1 = nn.Linear(64 * 7 * 7, 128) 
        self.fc2 = nn.Linear(128, 4) 
  
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.dropout1(x) 
        x = self.pool(F.relu(self.conv2(x))) 
        x = self.dropout2(x) 
        x = x.view(-1, 64 * 7 * 7) 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x) 
        return x
  

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda')

In [18]:
# Instantiate the model 
model = lassifier() 
  
# Move the model to the GPU if available 
model.to(device) 

lassifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [20]:
import torch.optim as optim 

In [42]:
xtr

tensor([[[[ 20,  20,  20],
          [ 27,  27,  27],
          [ 32,  32,  32],
          ...,
          [ 54,  54,  54],
          [ 56,  56,  56],
          [ 55,  55,  55]],

         [[ 19,  19,  19],
          [ 25,  25,  25],
          [ 31,  31,  31],
          ...,
          [ 54,  54,  54],
          [ 56,  56,  56],
          [ 56,  56,  56]],

         [[ 17,  17,  17],
          [ 23,  23,  23],
          [ 30,  30,  30],
          ...,
          [ 54,  54,  54],
          [ 55,  55,  55],
          [ 54,  54,  54]],

         ...,

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
         

In [21]:
# Define a loss function and optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
train_data = []
for i in range(len(x_data)):
   train_data.append([xtr[i], ytr[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=100)

In [26]:
BATCH_SIZE = 32
# initialize the train, validation, and test data loaders

# calculate steps per epoch for training and validation set
trainSteps = len(my_dataloader.dataset) // BATCH_SIZE
 

In [41]:
batch_size=32
num_epochs=10
 
losses = [] 
accuracies = [] 
  
# Train the model 
for epoch in range(1,num_epochs): 
    for i, (images, labels) in enumerate(my_dataloader): 
# Forward pass 
        images=images.to(device) 
        labels=labels.to(device) 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        
        # Backward pass and optimization 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

        _, predicted = torch.max(outputs.data, 1) 
    acc = (predicted == labels).sum().item() / labels.size(0) 
    accuracies.append(acc) 
    losses.append(loss.item()) 
        
    # Evaluate the model on the validation set 	 
    print('Epoch [{}/{}],Loss:{:.4f} ,Accuracy:{:.2f} '.format( 
        epoch+1, num_epochs, loss.item(),acc))


/tmp/ipykernel_42/2357366382.py:8: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for epoch in range(1,num_epochs):


RuntimeError: Input type (unsigned char) and bias type (float) should be the same

# Pneumonia merge

In [ ]:
 
directory = ['/kaggle/input/labeled-chest-xray-images/chest_xray/test/PNEUMONIA',
            '/kaggle/input/labeled-chest-xray-images/chest_xray/train/PNEUMONIA']
cnt = 0
# iterate over files in
# that directory
ids = 0
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)

        if os.path.isfile(f):
            cnt+=1
    x = data['Pneumonia']
    data.update({'Pneumonia': x+cnt})
    ids+=1
print(f'New Pneumonia {cnt}')
print(data)

In [ ]:
D = data

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
# # for python 2.x:
# plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
# plt.xticks(range(len(D)), D.keys())  # in python 2.x

plt.show()

# Covid Merge

In [ ]:
 
directory = ['/kaggle/input/covid-xray-dataset/DATA/DATA/1']
cnt = 0
 
for dirs in directory:
    cnt = 0
    for filename in os.listdir(dirs):
        f = os.path.join(dirs, filename)

        if os.path.isfile(f):
            cnt+=1
    
   
    x = data['COVID']
    data.update({'COVID': x+cnt})
    ids+=1
print(f'New Covid {cnt}')
print(data)

In [ ]:
show_bar(data)